In [1]:
import cv2
import numpy as np

def detect_line(frame):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    hsv_frame = cv2.cvtColor(cv2.cvtColor(blurred_frame, cv2.COLOR_GRAY2BGR), cv2.COLOR_BGR2HSV)

    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 30])

    mask = cv2.inRange(hsv_frame, lower_black, upper_black)

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(frame, [largest_contour], -1, (0, 0, 255), 2)

    #cv2.drawContours(frame, contours, -1, (0, 0, 255), 2)

    canny_frame = cv2.Canny(blurred_frame, 50, 150)

    lines = cv2.HoughLinesP(canny_frame, 1, np.pi/180, threshold=100, minLineLength=30, maxLineGap=10)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    return frame

In [2]:
cam = cv2.VideoCapture(0)
address = "http://192.168.1.5:8080/video"
cam.open(address)

if not cam.isOpened():
   print("Error opening camera!")
   exit()

while True:
    ret, frame = cam.read()
    
    if not ret:
        break

    cv2.imshow("Frame", frame)

    detected_line = detect_line(frame)

    cv2.imshow("Detected_Line", detected_line)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cam.release()
cv2.destroyAllWindows()